In [6]:
import kalman as kf
import filter 
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import os
import data_clean as dc 

In [3]:
#load in data
pvss = os.listdir("data/")
gps = pd.read_csv("data/"+pvss[0])
left_gps = pd.read_csv("data/"+pvss[3])
right_gps = pd.read_csv("data/"+pvss[1])
df_labels = pd.read_csv("data/"+pvss[2])

In [ ]:
label_names = pd.DataFrame(columns = ['road', 'condition','quality_right', 'quality_left','bumps'])

road_classes = ['dirt_road', 'cobblestone_road', 'asphalt_road']
quality_left_classes = ['good_road_left', 'regular_road_left', 'bad_road_left']
quality_right_classes = ['good_road_right', 'regular_road_right', 'bad_road_right']
condition_classes = ['paved_road', 'unpaved_road']
bump_classes=['no_speed_bump', 'speed_bump_asphalt', 'speed_bump_cobblestone']

# Convert from one-hot encoding to single label encoding
label_names = dc.ohe_to_label(df_labels, road_classes, label_names, 'road')
label_names = dc.ohe_to_label(df_labels, quality_right_classes, label_names, 'quality_right')
label_names = dc.ohe_to_label(df_labels, quality_left_classes, label_names, 'quality_left')
label_names = dc.ohe_to_label(df_labels, condition_classes, label_names, 'condition')
label_names = dc.ohe_to_label(df_labels, bump_classes, label_names, 'bumps')

# Convert road quality labels to numeric values
label_names = label_names.replace({'quality_right' : { 'good_road_right' : 2, 'regular_road_right' : 1, 'bad_road_right' : 0 }})
label_names = label_names.replace({'quality_left' : { 'good_road_left' : 2, 'regular_road_left' : 1, 'bad_road_left' : 0 }})
label_names['quality'] = label_names.loc[: , "quality_right":"quality_left"].mean(axis=1)
label_names = label_names.drop(columns = ["quality_right","quality_left"], axis = 1)

label_names.head()

In [ ]:
gps = dc.clean_gps(gps)
left_gps = dc.clean_acc(left_gps)
right_gps = dc.clean_acc(right_gps)

In [5]:
print(gps)

         timestamp   latitude  longitude   elevation  accuracy    bearing  \
0     1.577219e+09 -27.717812 -51.098895  948.770836      24.0  159.73294   
1     1.577219e+09 -27.717818 -51.098840  970.378820      12.0        NaN   
2     1.577219e+09 -27.717832 -51.098871  989.374267       4.0        NaN   
3     1.577219e+09 -27.717833 -51.098867  988.439139       4.0        NaN   
4     1.577219e+09 -27.717835 -51.098873  987.668730       4.0        NaN   
...            ...        ...        ...         ...       ...        ...   
1462  1.577220e+09 -27.717778 -51.099069  980.865571       4.0  130.51640   
1463  1.577220e+09 -27.717773 -51.099072  981.469505       6.0  130.51602   
1464  1.577220e+09 -27.717773 -51.099078  979.980588       6.0  194.52164   
1465  1.577220e+09 -27.717777 -51.099073  979.903556       6.0  195.58694   
1466  1.577220e+09 -27.717777 -51.099070  979.779629       6.0  195.58623   

      speed_meters_per_second  satellites provider  hdop  vdop  pdop  \
0  

In [ ]:
sep = 100
end = 20000

left_x = left_gps["acc_x_above"][:end][::sep]
right_x = right_gps["acc_x_above"][:end][::sep]
 
Q = 100*np.eye(4)   #100 represents our time step 
R = 5000*np.eye(2)

# state vector = [sx, sy, Vx, Vy] 